In [35]:
# encoding=utf8  
from PIL import ImageGrab
import time
from aip import AipOcr
import urllib
from urllib.parse import quote
import requests
import webbrowser

In [36]:
""" 读取图片 """
def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read()

In [231]:
PATH = lambda p: os.path.abspath(p)

def cap():
    # 每抓取一次屏幕需要的时间约为1s,如果图像尺寸小一些效率就会高一些
    path=time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
    debug = False
    # 截图的尺寸
    img = ImageGrab.grab(bbox=(90, 400, 1200-60, 1300)) 
    img.save(path+".png")
    return path+".png"
    #img = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)

In [38]:
""" 你的 APPID AK SK 打开下面链接，在百度云注册获取"""
"""https://cloud.baidu.com/doc/OCR/OCR-iOS-SDK/24.5C.E6.95.B0.E6.8D.AE.E6.8E.A5.E5.8F.A3.html"""
APP_ID = '替换自己申请的 APP_ID'
API_KEY = '替换自己申请的 API_KEY'
SECRET_KEY = '替换自己申请的 SECRET_KEY'

def getResult(filename):
    client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

    image = get_file_content(filename)

    """ 调用通用文字识别, 图片参数为本地图片 """
    result=client.basicGeneral(image)
    # result="{u'log_id': 578283474979346368, u'words_result_num': 9, u'words_result': [{u'words': u'\u6253\u65b0\u7684\u6807\u7b7e\u9875\u9996\u9875\u25a1\u516c\u53f8\u25a1RN\u25a1\u5b66\u4e60\u7f51\u7ad9\u25a1\u90e8\u7f72Qb'}, {u'words': u'\u4e3a\u82f1\u8bed\u538c'}, {u'words': u'\u76f2\u76ee\u81ea\u4fe1\u7fa4'}, {u'words': u'\u5f90\u5176\u4e1c'}, {u'words': u'2'}, {u'words': u'\u5468\u51e1:\u60f3\u4f4f\u8fd9\u6837\u4e00\u4e2a\u5c0f\u623f\u5b50\u662f\u591a'}, {u'words': u'3'}, {u'words': u'6. Studio Android Studio a##'}, {u'words': u'\u5e1d\u90fd-\u7f8e\u6ecbhttp:// www.apkbu'}]}"
    count=result.get("words_result_num")
    result_dic={"answer":None,"question":""}
    if count>3:
        answers=result.get("words_result")
        s=""
        ques_len=count
        ans_index=2
        for i in range(ques_len):
            words=answers[i].get("words")
            s+=words
            if words.find("?")>=0:
                ans_index=i+1
                break
        new_ans=answers[ans_index:]
        new_ans2=list()
        for i in range(len(new_ans)):
            new_ans2.append(new_ans[i].get("words"))
        result_dic["question"]=s
        result_dic["answer"]=new_ans2
    return result_dic


In [58]:
def start(path):
    result_dic = getResult(path)
    question = result_dic.get("question")
    answer = result_dic.get("answer")

    url = "http://m.baidu.com/s?wd=" + question
    url = quote(url, safe="/:?=")
    response=requests.get(url)
    content=response.text
    print(question)
    size = len(answer)
    for i in range(size):
        print("%s\t%d" % (answer[i], content.count(answer[i])))
    return url

In [265]:
#截屏
beg = time.time()
path = cap()
end = time.time()
print (beg - end)

# 开始识别
beg = time.time()
url = start(path)
end = time.time()
print (beg - end)

-0.5160949230194092
12.地壳中含量最多的金属元素是什么?
铝	8
铁	9
钙	3
-7.610493898391724


In [253]:
webbrowser.open(url, new=0, autoraise=True)

True